<a href="https://colab.research.google.com/github/gitgit991/Depression-Detection-/blob/main/CNN%20Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [1]:
!pip install --upgrade numpy gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.8 MB/s eta 0:00:00


In [17]:
from google.colab import drive
drive.mount('/content/drive/')
import os

# Change directory to your dataset location

import nltk
nltk.download('stopwords')
import numpy as np
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
import gc
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from smart_open import open
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from keras.layers import Dropout
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
# Load datasets
dataset1 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv', delimiter=',', encoding='utf-8'))[:, 0:2]
dataset2 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/full_test_split.csv', delimiter=',', encoding='utf-8'))[:, 0:2]
dataset3 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv', delimiter=',', encoding='utf-8'))[:, 0:2]

dataset = np.concatenate((dataset1, np.concatenate((dataset2, dataset3))))

def checkPosNeg(dataset, index):
    for i in range(len(dataset)):
        if dataset[i][0] == index:
            return dataset[i][1]
    return 0

# Initialize lists
Data, Y = [], []
Data_test, Y_test = [], []

# Process training data
for i in range(len(dataset3)):
    val = checkPosNeg(dataset, dataset3[i][0])
    Y.append(val)
    try:
        fileName = f"/content/drive/My Drive/MCA Dataset/train_data/{int(dataset3[i][0])}_TRANSCRIPT.csv"
        Data.append(np.array(pd.read_csv(fileName, delimiter='\t', encoding='utf-8', engine='python'))[:, 2:4])
    except Exception:
        continue

# Process validation data
for i in range(len(dataset1)):
    val = checkPosNeg(dataset, dataset1[i][0])
    Y.append(val)
    try:
        fileName = f"/content/drive/My Drive/MCA Dataset/dev_data/{int(dataset1[i][0])}_TRANSCRIPT.csv"
        Data.append(np.array(pd.read_csv(fileName, delimiter='\t', encoding='utf-8', engine='python'))[:, 2:4])
    except Exception:
        continue

# Process test data
for i in range(len(dataset2)):
    Y_test.append(checkPosNeg(dataset, dataset2[i][0]))
    try:
        fileName = f"/content/drive/My Drive/MCA Dataset/test_data/{int(dataset2[i][0])}_TRANSCRIPT.csv"
        Data_test.append(np.array(pd.read_csv(fileName, delimiter='\t', encoding='utf-8', engine='python'))[:, 2:4])
    except Exception:
        continue

# Convert lists to numpy arrays
Y = np.array(Y)
Y_test = np.array(Y_test)

Data2, Data2_test = [], []

# Extract participant's speech
for i in range(len(Data)):
    script = [Data[i][k][1] for k in range(1, len(Data[i])) if Data[i][k][0] == "Participant"]
    Data2.append(script)

for i in range(len(Data_test)):
    script = [Data_test[i][k][1] for k in range(1, len(Data_test[i])) if Data_test[i][k][0] == "Participant"]
    Data2_test.append(script)

# Free memory
Data, Data_test = [], []
gc.collect()

# Convert to numpy object arrays
Data2 = np.array(Data2, dtype=object)
Data2_test = np.array(Data2_test, dtype=object)

# Load word embeddings
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/MCA Dataset/GoogleNews-vectors-negative300.bin', binary=True)
stop_words = set(stopwords.words('english'))

# Function to remove stopwords
def remove_StopWords(sentence):
    return [w for w in sentence if w not in stop_words]

# Create feature matrices
max_num_words, max_num_sentence = 20, 250
finalMatrix = np.zeros((len(Data2), max_num_sentence, max_num_words, 300))
finalMatrix_test = np.zeros((len(Data2_test), max_num_sentence, max_num_words, 300))

# Convert words to embeddings for training data
for k in range(len(Data2)):
    for i in range(min(max_num_sentence, len(Data2[k]))):
        try:
            sentence = str(Data2[k][i])  # Convert to string to avoid float error
            sentence = remove_StopWords(sentence.split(" "))  # Remove stopwords
            for j in range(min(max_num_words, len(sentence))):
                try:
                    word = sentence[j].strip("<>").strip()  # Clean word
                    finalMatrix[k][i][j] = model[word]
                except KeyError:
                    continue  # Ignore words not found in embeddings
        except Exception as e:
            continue  # Skip problematic rows

# Convert words to embeddings for test data
for k in range(len(Data2_test)):
    for i in range(min(max_num_sentence, len(Data2_test[k]))):
        try:
            sentence = str(Data2_test[k][i])  # Convert to string
            sentence = remove_StopWords(sentence.split(" "))
            for j in range(min(max_num_words, len(sentence))):
                try:
                    word = sentence[j].strip("<>").strip()
                    finalMatrix_test[k][i][j] = model[word]
                except KeyError:
                    continue
        except Exception:
            continue


# Free memory
Data2, Data2_test, model, stop_words = [], [], [], []
gc.collect()

# Function to upsample data
def upsample(X_train, Y_train):
    X_train = np.array(X_train)
    Y_train = np.array(Y_train)

    print("Before upsampling: ", X_train.shape, Y_train.shape)

    if len(X_train) != len(Y_train):
        print(f"Mismatch: X_train has {len(X_train)} samples, Y_train has {len(Y_train)} labels")
        Y_train = Y_train[:len(X_train)]  # Trim labels if needed

    X_train_0 = X_train[Y_train == 0]
    X_train_1 = X_train[Y_train == 1]

    Y_train_1 = Y_train[Y_train == 1]
    size = X_train_0.shape[0] - X_train_1.shape[0]

    X_train, Y_train = list(X_train), list(Y_train)

    while size > 0:
        size -= 1
        index = np.random.randint(0, X_train_1.shape[0] - 1)
        leave_index = np.random.randint(0, len(X_train) - 1)
        X_add, X_leave = X_train_1[index], X_train[leave_index]
        Y_add, Y_leave = Y_train_1[index], Y_train[leave_index]

        X_train[leave_index] = X_add
        X_train.append(X_leave)
        Y_train[leave_index] = Y_add
        Y_train.append(Y_leave)

    return np.array(X_train), np.array(Y_train)

# **Fix shape mismatch before upsampling**
if len(Y) != finalMatrix.shape[0]:
    print(f"⚠️ Warning: Shape mismatch - Adjusting Y from {len(Y)} to {finalMatrix.shape[0]}")
    Y = Y[:finalMatrix.shape[0]]

if len(Y_test) != finalMatrix_test.shape[0]:
    print(f"⚠️ Warning: Shape mismatch - Adjusting Y_test from {len(Y_test)} to {finalMatrix_test.shape[0]}")
    Y_test = Y_test[:finalMatrix_test.shape[0]]

# **Convert Y to numpy array**
Y = np.array(Y, dtype=np.int32)
Y_test = np.array(Y_test, dtype=np.int32)

# **Run upsampling**
finalMatrix, Y = upsample(finalMatrix, Y)
finalMatrix_test, Y_test = upsample(finalMatrix_test, Y_test)

print("Final shape after upsampling:")
print("Train:", finalMatrix.shape, Y.shape)
print("Test:", finalMatrix_test.shape, Y_test.shape)

⚠️ Warning: Shape mismatch - Adjusting Y from 142 to 37
⚠️ Warning: Shape mismatch - Adjusting Y_test from 47 to 14
Before upsampling:  (37, 250, 20, 300) (37,)
Before upsampling:  (14, 250, 20, 300) (14,)
Final shape after upsampling:
Train: (40, 250, 20, 300) (40,)
Test: (16, 250, 20, 300) (16,)


In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input

class CNN_Text:
    def __init__(self):
        classifier = Sequential()
        classifier.add(Input(shape=(finalMatrix.shape[1], finalMatrix.shape[2], finalMatrix.shape[3])))
        classifier.add(Conv2D(150, (1, 5), activation='relu'))
        classifier.add(MaxPooling2D(pool_size=(1, 3)))
        classifier.add(Conv2D(75, (1, 3), activation='relu'))
        classifier.add(MaxPooling2D(pool_size=(1, 2)))
        classifier.add(Flatten())
        classifier.add(Dense(units=128, activation='relu'))
        classifier.add(Dense(units=1, activation='sigmoid'))
        classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        self.classifier = classifier  # Corrected spelling

    def fitModel(self, X, Y, epoch=10):
        class_weight = {0: 0.5, 1: 0.6}
        self.classifier.fit(X, Y, epochs=epoch, class_weight=class_weight)

    def predictModel(self, X):
        return Thresholding(self.classifier.predict(X), 0.5)

# Initialize and train model
model = CNN_Text()
model.fitModel(finalMatrix, Y, 10)
Y_Pred = model.predictModel(finalMatrix_test)
print(classification_report(Y_test, Y_Pred))


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 751ms/step - accuracy: 0.5000 - loss: 0.3899
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 1.0000 - loss: 0.2457
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 705ms/step - accuracy: 1.0000 - loss: 0.1146
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 717ms/step - accuracy: 1.0000 - loss: 0.0402
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 1.0000 - loss: 0.0176
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 699ms/step - accuracy: 1.0000 - loss: 0.0040
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 1.0000 - loss: 0.0018
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 726ms/step - accuracy: 1.0000 - loss: 2.8257e-04
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 1.0000 - loss: 1.0630e-04
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 740ms/step - accuracy: 1.0000 - loss: 8.8519e-05
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 715ms/step
              precision    recall  f1-score   support

           0       0.64      0.88      0.74         8
           